In [ ]:
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

import os
import pandas as pd
from tqdm import *
from scipy import stats

from pathlib import Path

NathanPath="d:\Documents\Info\INF554\INF554_Kaggle_Project"
NathanPath="/users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project"

In [ ]:
project_path = str(Path(os.getcwd()).parent.absolute())
print("Current directory : " + os.getcwd() + ", Project directory : " + project_path)

In [ ]:
os.chdir(project_path)
os.chdir(NathanPath)

In [ ]:

# Read training data
df_train = pd.read_csv('data/train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# Read test data
df_test = pd.read_csv('data/test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

df_dummy_sub = pd.read_csv('submissions/dummy_submission.csv', dtype={'author': np.int64, 'hindex': np.float32})

In [ ]:
G = nx.read_edgelist('data/coauthorship.edgelist', delimiter=' ', nodetype=int)
n=G.number_of_nodes()
print("Number of nodes : " + str(n))

abs_nodeID_Graph=dict(enumerate(G.nodes))
nodeID_abs_Graph=dict([(b,a) for a,b in enumerate(G.nodes)])

In [ ]:
abs_nodeID_Train=dict(df_train["author"])
abs_hindex_Train=dict(df_train["hindex"])
nodeID_abs_Train=dict([(b,a) for a,b in abs_nodeID_Train.items()])

abs_nodeID_Test=dict(df_test["author"])
abs_hindex_Test=dict(df_test["hindex"])
nodeID_abs_Test=dict([(b,a) for a,b in abs_nodeID_Test.items()])

In [ ]:
"""train_dict=[(a,b) for a,b in np.array(df_train, dtype=np.int64)]
id_train=[i for i,_ in train_dict]
id_test=[x for x,_ in np.array(df_test, dtype=np.int64)]

idx = np.random.permutation(len(id_train))

idx_train_indexes=idx[:int(0.8*len(id_train))]
idx_train = [id_train[i] for i in idx_train_indexes]
idx_val_indexes=idx[int(0.8*len(id_train)):]
idx_val = [id_train[i] for i in idx_val_indexes]



G_train=G.subgraph(id_train)
h_index_train=[x for _,x in train_dict]

print(len(idx_train))
print(len(idx_val))

h_index_trainx=[h_index_train[x] for x in idx[:int(0.8*len(id_train))]]
h_index_valx=[h_index_train[x] for x in idx[int(0.8*len(id_train)):]]"""

In [ ]:
def random_walk(G, node, walk_length):
    # Simulates a random walk of length "walk_length" starting from node "node"
    walk=[]
    for _ in range(walk_length):
        node=np.random.choice(list(G.neighbors(node)))
        walk.append(node)
    walk = [node for node in walk]
    return walk

In [ ]:
def generate_walks(G, num_walks, walk_length):
    # Runs "num_walks" random walks from each node
    walks = []
    for x in G.nodes():
        for _ in range(num_walks):
            walks.append(random_walk(G,x,walk_length))
    np.random.shuffle(walks)
    return walks

In [ ]:
def deepwalk(G, num_walks, walk_length, n_dim):
    # Simulates walks and uses the Skipgram model to learn node representations
    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)
    print("Training word2vec")
    model = Word2Vec(vector_size=n_dim, window=8, min_count=0, sg=1, workers=8, hs=1)
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)
    return model

n_dim = 128
n_walks = 10
walk_length = 10
#model = deepwalk(G, n_walks, walk_length, n_dim) 
#model.save("DeepWalk/Models/model_"+str(n_walks)+'_'+str(walk_length)+'_'+str(n_dim))

In [ ]:
#model=Word2Vec.load("DeepWalk/Models/model_"+str(n_walks)+'_'+str(walk_length)+'_'+str(n_dim))
model=Word2Vec.load("DeepWalk/Models/model_10_10_128")

In [ ]:
print(len(model.wv), G.number_of_nodes())
list(model.wv)

In [ ]:
DeepWalk_embeddings = np.empty(shape=(n, n_dim))

print("Filling embeddings")
for i in range(G.number_of_nodes()):
    try:
        DeepWalk_embeddings[i]=model.wv[i]
    except:
        pass


In [ ]:
def RSE(X,Y):
    return (X-Y)@(X-Y) / len(X)

"""X_train_x = [DeepWalk_embeddings[i] for i in idx_train_indexes]
X_validate_x = [DeepWalk_embeddings[i] for i in idx_val_indexes]

lin_reg=LinearRegression()
lin_reg.fit(X_train_x,h_index_trainx)
_pred=lin_reg.predict(X_validate_x)
score=RSE(h_index_valx,_pred)


print("Accuracy using DeepWalk embeddings ", score)"""